# Systematic Experimentation (with Ansible)
<i>Adapted for use with FABRIC from [Systematic Experimentation (with Ansible)](https://groups.geni.net/geni/wiki/GENIExperimenter/Tutorials/SystematicExperimentationAnsible)

<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
  
This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)
<img src="./Figures/SETop.png">

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'SystematicExperimentationWithAnsible'
try:
    slice = fablib.get_slice(slice_name)   
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Guieded Experiment

### 2.1 Download the Ansible playbook
Download the Ansible playbook, webpages, etc needed to configure the nodes:
1. Open a Terminal and login to the ansible node.
2. Create a folder to save the information
<br> ``` mkdir ansible```
3. move inside of the folder you created and download the ansible playbook tarfile from the geni website
<br> ``` cd ansible```
<br> ``` wget https://github.com/GENI-NSF/geni-tutorials/raw/master/ansible/ansible.tar.gz ```
4. de-compress the tarfile to extract the playbook information
<br> ``` tar zxvf ansible.tar.gz```

### 2.2 Configure and initialize
To start ansible you will have to follow some steps to get the connection set up
1. Take a look at your Ansible inventory file: 
<br> on your Ansible node open a Terminal and run the following command to see the available nodes (make sure you are in the home directory):
<br> ```$ cat inventory```
2. Be sure your ```private key``` has been added to your ```SSH agent```:
<br> ```ssh-agent bash```
<br> ```ssh-add ~/Ansible_Key```
3. Check to see if your nodes are up and ready. 
<br> ```$ ansible -i inventory all -m ping ```
<br> the output of the command should look like this:
<br> ```   server | SUCCESS => {                                      |```
<br> ```    "ansible_facts":                                          |```
<br> ```        "discovered_interpreter_python": "/usr/bin/python3"   |```
<br> ```    },                                                        |```
<br> ```    "changed": false,                                         |```
<br> ```    "ping": "pong"                                            |```
<br> ``` }                                                            |```
<br> ``` client | SUCCESS => {                                        |```
<br> ```     "ansible_facts": {                                       |```
<br> ```         "discovered_interpreter_python": "/usr/bin/python3"  |```
<br> ```     },                                                       |```
<br> ```    "changed": false,                                         |```
<br> ```   "ping": "pong"                                             |```
<br> ``` }                                                            |```
<br> be aware that since it is the first time you are using ssh on the nodes they might requiere you to fingerprint, just type ```yes``` untill all of the fingerprint request end
5. Try using the ping module in Ansible to only ping ```server_1``` or ```host_1``` by replacing all in the above with ```server_1``` or ```host_1```.

### 2.2 Ansible Playbook
<br>Ansible commands can be collected into files called Playbooks. Playbooks are in a configuration file format called YAML which is very straightforward. In particular, Ansible Ad Hoc commands easily map to commands used in an Ansible Playbook.

The Playbook to configure the ```server``` node is in ```server.yml```. It links to other files. For example, the code to tell the ```server``` node to run an ```nmap``` scan and post the results is in ```roles/nmap/tasks/map.yml``` and looks as follows:
<br>```   - name: map network using nmap                                                                          |```
<br>```     command: nmap -sP -oX {{ nmap_xml_file }} {{ address_range }}                                         |```
<br>```   - name: convert nmap xml to html                                                                        |```
<br>```     shell:  xsltproc /usr/share/nmap/nmap.xsl {{ nmap_xml_file }} > {{ nmap_html_file }}                  |```
<br>```   - name: create directory for nmap logs in WEB_ROOT/nmaplogs with permissions of 755                     |```
<br>```     file: >                                                                                               |```
<br>```        dest={{ WEB_ROOT }}/{{ nmap_dir }}                                                                 |```
<br>```        state=directory                                                                                    |```
<br>```        mode=755                                                                                           |```
<br>```   - name: copy nmap html file to a public place                                                           |```
<br>```     command: mv {{ nmap_html_file }} {{ WEB_ROOT }}/{{ nmap_dir }}/nmap.html removes={{ nmap_html_file }} |```

### 2.3 Exercise
#### 2.3.1 Configure the nodes
1. Edit the file ```group_vars/all.yml``` so that the ```address_range variable``` uses the IP subnet from your topology. It should look similar to this:
<br>```address_range: 10.10.1.1-10```
2. Run the playbook to configure the ```server``` with the following command on the local machine:
<br>```ansible-playbook server.yml -i inventory --limit server_0```
3. Browse to hostname of the server node in your browser. Click on the ```nmap``` link.
4. If this looks ok, run the following to install the code on both of your servers:
<br>```ansible-playbook server.yml -i inventory```
5. Browse to the hostname of the ```server-0``` node in your browser.
<br> one way to acomplish this step is to open a client terminal and using ```wget``` to see if you get the "index.html" file

#### 2.3.2 Update a portion of the configuration
1. Run the following command to only update the ```nmap``` portion of the ```server_0``` configuration:
<br> ```ansible-playbook update-map.yml -i inventory```
2. Feel free to change the value of ```address_range``` in ```groups_vars/all.yml``` and rerun ```update-map.yml``` to search for different nodes. 

## Cleanup Resources

Once you have completed the assignment shut down the slice


In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")